In [1]:
# Copyright 2019-2020, ETH Zurich, Media Technology Center
#
# This file is part of Federated Learning Project at MTC.
#
# Federated Learning is a free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Federated Learning is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser Public License for more details.
#
# You should have received a copy of the GNU Lesser Public License
# along with Federated Learning.  If not, see <https://www.gnu.org/licenses/>.
"""
The Python implementation of the MTC Federated Learning Example Operator.

This is an example how to use the Operator Class.

Look at operator_class_db.py for all callable functions.

The example creates a model and runs 10 rounds of send_model_to_nodes->training_on_nodes->
send_back_to_server->aggregate_gradients_to_get_new_global_models

"""
import os

import sys
if os.getcwd().split('/')[-1]!="Federated-Learning":
    os.chdir('../..')
print(os.getcwd())
sys.path.append(os.getcwd())
os.environ['STATIC_VARIABLES_FILE_PATH'] = "globalserver/static_variables.json"

os.environ['PATH_TO_GLOBALSERVER'] = "globalserver/api/"
import tensorflow as tf
import xgboost as xgb
import pandas as pd
import logging
from globalserver.operator_.operator_class_db import Operator
from testing.test_class import Testing
from globalserver.operator_.utils import operator_utils
import json

tf.keras.backend.clear_session()
clients = ['r0', 'r1']
TestSetup = Testing(clients, start_servers=True, clear_logs=True, clear_db=False, interface=False)



/home/schneebi/PycharmProjects/federated-learning/Federated-Learning


2020-05-15 10:51:12,148 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / kill_servers / 194] KILLING SERVERS
2020-05-15 10:51:12,150 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / kill_global_server / 156] KILLING GLOBAL SERVER
2020-05-15 10:51:12,153 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / kill_client_interface_node / 163] KILLING CLIENT INTERFACE NODE
2020-05-15 10:51:12,155 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / kill_client_interface / 173] KILLING CLIENT INTERFACE
2020-05-15 10:51:12,157 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / kill_node_servers / 183] KILLINGCLIENTS
2020-05-15 10:51:15,312 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / start_global_server / 337] STARTING UP GLOBAL SERVER
2020-05-15 10:51:15,321 [MainProcess ] [INFO ] [OPERATOR] [test_class.py / start_node_servers / 225] STARTING UP NODES


the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'api/globalserver_task_controller.py', 'C88A33B946']
the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'node_task_controller.py', 'r0', 'C88A33B946']
the commandline is ['/home/schneebi/PycharmProjects/federated-learning/venv/bin/python', 'node_task_controller.py', 'r1', 'C88A33B946']


In [2]:
specification = {
    'n_features': 2,  # for given test data
    'feature_information': {  # can list up categorical features and initialize them with False

    },
    'n_estimators': 2,  # number of trees in the final forest
    'max_depth': 5,  # maximum depth parameter
    'max_features': 2,  # maximum features to consider at each node for splitting
    'max_bins': 20,  # maximum number of bins to use for continuous data in the histograms
    'pos_label': 1,  # positive label in data
    'neg_label': 0,  # negative label in data
    'minimal_information_gain': 0.0,  # minimal information gain to not add a leaf
    'metrics': ['log_loss'],
    "preprocessing": {
        "noise": {
            "epsilon": 1,
            "delta": 0.2
        }
    }
}

get_compiled_model_RF = operator_utils.RF_get_compiled_model

In [3]:
setup_dict = {"model_function": {
    "function": get_compiled_model_RF,
    "parameters": specification,
},
    "protocol": 'RF',
    "model_name": "test_model",
    "model_description": "this model is just to test the db",
    "testing": True,
    "training_config": {
                        "diff_p_epsilon": 10.0,
                        "diff_p_delta": 0.9,
                        "steps_per_epoch": 1,
                        "batch_size": 2000,
                        "dataset": "",
                        "skmetrics": ["f1_score", "accuracy_score"],
                        "tfmetrics": ["AUC", "Accuracy"]},
    "rounds": pow(2, (specification['max_depth'] + 1)) - 1,
    "round": ["fetch_model", "train_model", "send_model", "aggregate"],
    "final_round": [],
    "clients": clients,
    "experiment_name": "kkbox",
    "experiment_description": f"desc if nice experiment",
    "preprocessing": {
        "noise": {
            "epsilon": 1,
            "delta": 0.2
        },
    }
}

Define one Experiment for each tree

In [4]:
experiment_id_list = []
operator_list = []
for _ in range(specification['n_estimators']):
    # it's important that each tree is specified individually!
    # model_dict, empty_forest_dict = operator_utils.RF_get_compiled_model(specification)
    operator_i = Operator()
    operator_list.append(operator_i)
    experiment_id = operator_i.setup_up_experiment_by_dict(setup_dict, additional_description='')
    experiment_id_list.append(experiment_id)

2020-05-15 10:51:16,124 [MainProcess ] [DEBUG] [OPERATOR] [cmd.py / execute / 719] Popen(['git', 'version'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=None)
2020-05-15 10:51:16,159 [MainProcess ] [DEBUG] [OPERATOR] [cmd.py / execute / 719] Popen(['git', 'version'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=None)
2020-05-15 10:51:16,172 [MainProcess ] [DEBUG] [OPERATOR] [cmd.py / execute / 719] Popen(['git', 'cat-file', '--batch-check'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=<valid stream>)
2020-05-15 10:51:16,352 [MainProcess ] [DEBUG] [OPERATOR] [cmd.py / execute / 719] Popen(['git', 'cat-file', '--batch-check'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=<valid stream>)


Start master experiment

In [5]:

operator = Operator()
# initialize same for overhead process
setup_dict['rounds']=0
setup_dict['final_round']=["fetch_model", "send_training_loss", "send_validation_loss", "send_test_loss"]
experiment_id = operator.setup_up_experiment_by_dict(setup_dict, additional_description='')

"""
Overhead process:
- constructs the whole random forest
- after construction is finished, the losses from all agents get acquired and stored in the database
"""
# start experiment
operator.start_experiments_RF(
    experiment_id=experiment_id,
    experiment_id_list=experiment_id_list,
    operator_list=operator_list,
    idle_time=1,
    n_workers=1
)

2020-05-15 10:51:16,541 [MainProcess ] [DEBUG] [OPERATOR] [cmd.py / execute / 719] Popen(['git', 'cat-file', '--batch-check'], cwd=/home/schneebi/PycharmProjects/federated-learning/Federated-Learning, universal_newlines=False, shell=None, istream=<valid stream>)
/home/schneebi/PycharmProjects/federated-learning/venv/lib/python3.6/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/home/schneebi/PycharmProjects/federated-learning/venv/lib/python3.6/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
2020-05-15 10

2020-05-15 10:51:39,742 [Process-1   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:51:39,743 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe1a8'), 'task_order': 11, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:51:39,746 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:51:39,755 [Process-2   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.008296710150543674
2020-05-15 10:51:39,757 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe19c
2020-05-15 10:51:39,758 [Process-1   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.008296710150543674
2020-05-15 10:51:39,760 [Process-1   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated hist

2020-05-15 10:51:52,885 [Process-1   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:51:52,887 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:51:52,894 [Process-1   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.0
2020-05-15 10:51:52,896 [Process-2   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.0
2020-05-15 10:51:52,902 [Process-1   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe09c
2020-05-15 10:51:52,902 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe19c
2020-05-15 10:51:53,914 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe0b5'), 'task_order': 24, 'task_name': 'fetch_model', 'task_status': 'scheduled'}
2020-

2020-05-15 10:52:06,027 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe1c0'), 'task_order': 35, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:52:06,030 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:52:06,043 [Process-2   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.011859896560208472
2020-05-15 10:52:06,048 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe19c
2020-05-15 10:52:07,032 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe0c2'), 'task_order': 37, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-05-15 10:52:07,058 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': Obj

2020-05-15 10:52:19,158 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:52:19,163 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe0ce'), 'task_order': 49, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-05-15 10:52:19,165 [Process-2   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.008098341271272735
2020-05-15 10:52:19,167 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe19c
2020-05-15 10:52:20,167 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe0cf'), 'task_order': 50, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:52:20,173 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': Object

...

2020-05-15 10:55:08,335 [Process-1   ] [INFO ] [OPERATOR] [operator_class_db.py / RF_aggregate / 489] Aggregated histograms5ebe58046d77a3b1253fe09c
2020-05-15 10:55:09,107 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe26f'), 'task_order': 210, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:09,352 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe16d'), 'task_order': 208, 'task_name': 'fetch_model', 'task_status': 'scheduled'}
2020-05-15 10:55:10,113 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe270'), 'task_order': 211, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:55:10,116 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:55:10,124

2020-05-15 10:55:21,775 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe179'), 'task_order': 220, 'task_name': 'fetch_model', 'task_status': 'scheduled'}
2020-05-15 10:55:22,272 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe27b'), 'task_order': 222, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:22,781 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe17a'), 'task_order': 221, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-05-15 10:55:23,275 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe27c'), 'task_order': 223, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:55:23,277 [Process-2   ] [INFO ] 

2020-05-15 10:55:34,933 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe186'), 'task_order': 233, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-05-15 10:55:35,365 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe287'), 'task_order': 234, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:35,936 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe187'), 'task_order': 234, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:36,370 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe288'), 'task_order': 235, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:55:36,371 [Process-2   ] [INFO ] [

2020-05-15 10:55:47,467 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe293'), 'task_order': 246, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:48,052 [Process-1   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe193'), 'task_order': 246, 'task_name': 'send_model', 'task_status': 'scheduled'}
2020-05-15 10:55:48,471 [Process-2   ] [DEBUG] [OPERATOR] [operator_utils.py / get_current_task / 96] Working on {'task_id': ObjectId('5ebe58046d77a3b1253fe294'), 'task_order': 247, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-05-15 10:55:48,472 [Process-2   ] [INFO ] [OPERATOR] [operator_class_db.py / aggregate / 254] Aggregating...
2020-05-15 10:55:48,475 [Process-2   ] [INFO ] [OPERATOR] [operator_utils.py / RF_aggregate_model_updates / 534] 0.0
2020-05-15 10:55:48,478 [Process-2   ] [INFO ] [OPERATO

True